In [ ]:
# Dependencies
import requests
import json
import pandas as pd
from config import api_key
from pandas.io.json import json_normalize  

## Get Scopus Author Information using Multiple Scopus Author IDs

In [ ]:
# File to Load
radiation_oncology = "radiation_oncology.csv"

# Read the CSV file and store into Pandas DataFrame with the column Scopus Author ID as a string
radiation_oncology_df = pd.read_csv(radiation_oncology, encoding="utf-8",dtype={'Scopus Author ID': str})

#Change the column names to lower case with underscore for spaces
radiation_oncology_df.columns =  radiation_oncology_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","")
radiation_oncology_df.head()

In [ ]:
#List the column names in the dataframe
radiation_oncology_df.columns

In [ ]:
#Check the data type in the dataframe columns called scopus_author_id and scopus_search
radiation_oncology_df.scopus_author_id.dtype
radiation_oncology_df.scopus_search.dtype

In [ ]:
#Save the column called scopus_author_id to a list called Author_ID_List
author_ID_List = radiation_oncology_df['scopus_author_id'].tolist()
print(author_ID_List)

In [ ]:
#Clean the Author_ID_List to remove nan
cleaned_Author_ID_List = [x for x in author_ID_List if str(x) != 'nan']
print(cleaned_Author_ID_List)

In [ ]:
scopus_Search_List = radiation_oncology_df['scopus_search'].tolist()
print(scopus_Search_List)

In [ ]:
cleaned_Scopus_Search_List = [x for x in scopus_Search_List if str(x) != 'nan']
print(cleaned_Scopus_Search_List)

In [ ]:
def get_scopus_articles(cleaned_Scopus_Search_List):
    for authid in cleaned_Scopus_Search_List:
        url = "http://api.elsevier.com/content/search/scopus?"
        headers = {
             "X-ELS-APIKey": api_key,
             'Accept':'application/json'
        }
        parameters = {
            "query": authid,
            "field": "eid"
        }
        try:
            multiple_author_response = requests.get(url, headers=headers, params=parameters)
            print(multiple_author_response.url)
            print(multiple_author_response.status_code)
            response_multiple_author_search = multiple_author_response.json()
            response_multiple_author_search_cleaned = json.dumps( response_multiple_author_search, indent=4, sort_keys=True)
            print(response_multiple_author_search_cleaned)
        except:
            print("Could not use" + authid)
    return(response_multiple_author_search_cleaned)
get_scopus_articles(cleaned_Scopus_Search_List)